In [3]:
import requests
import json
import time

def create_pushshift_url(subreddit, from_ts, to_ts, size):
    url = 'https://api.pushshift.io/reddit/search/submission/?&size='+str(size)+'&after='+str(from_ts)+'&before='+str(to_ts)+'&subreddit='+str(subreddit)+'&fields=title,selftext,id,created_utc'
    return url

def get_data(subreddit, from_date, to_date, size, text_length):
    from_ts = int(time.mktime(from_date.timetuple()))
    to_ts = int(time.mktime(to_date.timetuple()))
    data = list()
    while len(data) < size:
        url = create_pushshift_url(subreddit, from_ts, to_ts, 500)
        r = requests.get(url)
        data_part = json.loads(r.text)['data']
        data_part = [item for item in data_part if 'selftext' in item and 'created_utc' in item and len(item['selftext']) < text_length and item['selftext'] != '[removed]' and item['selftext'] != '[deleted]']
        data_part = list(map(lambda item: {'id': item['id'], 'text': ' '.join([' '.join(item['title'].split()), ' '.join(item['selftext'].split())]), 'time': item['created_utc']}, data_part))
        data = data + data_part
        from_ts = data[-1]['time'] + 1
        print("got {} items ({} total), next date: {}".format(len(data_part), len(data), from_ts))
    return data

In [4]:
from datetime import date
from_date = date.fromisoformat('2019-05-01')
to_date = date.fromisoformat('2020-01-01')

data = get_data("AskReddit", from_date, to_date, 2000, 100)


got 403 items (403 total), next date: 1556663784
got 388 items (791 total), next date: 1556666041
got 403 items (1194 total), next date: 1556668050
got 390 items (1584 total), next date: 1556670437
got 400 items (1984 total), next date: 1556672828
got 387 items (2371 total), next date: 1556675143


In [6]:
data[20]

{'id': 'bj9bfv',
 'text': 'You’re able to swap the smell of two things, what do you swap? ',
 'time': 1556661695}

In [7]:
id = 0
errored = []
for submission in data:
    print("processing " + str(id))
    url = 'https://api.pushshift.io/reddit/comment/search/' + '?link_id=' + submission['id'] + '&size=10' +  '&nest_level=1' + '&fields=id,body'
    try:
        r = requests.get(url)
        comments = json.loads(r.text)['data']
        submission['comments'] = [ c for c in [' '.join(c['body'].split()) for c in comments] if len(c) < 100]
    except:
        print("errored " + str(id))
        errored.append(id)
    id += 1

processing 1121
processing 1122
processing 1123
processing 1124
processing 1125
processing 1126
processing 1127
processing 1128
processing 1129
processing 1130
processing 1131
processing 1132
processing 1133
processing 1134
processing 1135
processing 1136
processing 1137
processing 1138
processing 1139
processing 1140
processing 1141
processing 1142
processing 1143
processing 1144
processing 1145
processing 1146
processing 1147
processing 1148
processing 1149
processing 1150
processing 1151
processing 1152
processing 1153
processing 1154
processing 1155
processing 1156
processing 1157
processing 1158
processing 1159
processing 1160
processing 1161
processing 1162
processing 1163
processing 1164
processing 1165
processing 1166
processing 1167
processing 1168
processing 1169
processing 1170
processing 1171
processing 1172
processing 1173
processing 1174
processing 1175
processing 1176
processing 1177
processing 1178
processing 1179
processing 1180
processing 1181
processing 1182
processi

In [10]:
data[302]

{'id': 'bj9mgd',
 'text': "What convinces you that true love does or doesn't exist? ",
 'time': 1556663203,
 'comments': ['Does, and the amazing woman sitting next to me!',
  "You know, if she believed in me— I'd find a way."]}

In [11]:
import csv
def create_csv(data, filename):
    upload_count = 0
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["ID","Message","Comment"]
        a.writerow(headers)
        for sub in data:
            if 'comments' not in sub:
                continue
            for c in sub['comments']:
                a.writerow([sub['text'], c])

In [12]:
create_csv(data, '../data/reddit_ask_v2.csv')